### 1. Importy


In [1]:
# 1) Importy
from pathlib import Path
import hashlib, json, shutil, time
from PIL import Image
from ipywidgets import Dropdown, Checkbox, IntText, Button, HBox, VBox
from IPython.display import display
from csv import DictWriter
import json, shutil, hashlib
from pathlib import Path

# 1) Parametry
RUN_ID = time.strftime("%Y%m%d_%H%M%S_A1")
MIN_W, MIN_H = 256, 256  # filtr jakości
ROOT = Path.cwd()
DIR_INPUTS = ROOT / "inputs"
DIR_RAW = ROOT / "data" / "raw"
DIR_STAGING = ROOT / "data" / "staging" / RUN_ID
DIR_PREV = ROOT / "outputs" / "previews" / "ingest" / RUN_ID
DIR_STAGING.mkdir(parents=True, exist_ok=True)
DIR_PREV.mkdir(parents=True, exist_ok=True)

# 2) UI wyboru folderu źródłowego
candidates = [DIR_INPUTS] + [p for p in (DIR_RAW.glob("*")) if p.is_dir()]
dd_src = Dropdown(options=candidates, description="Źródło:")
cb_thumbs = Checkbox(value=True, description="Miniatury")
it_max = IntText(value=500, description="Max plików")
btn = Button(description="Start ingest")
display(VBox([HBox([dd_src, it_max]), cb_thumbs, btn]))

### 2. Funkcje pomocnicze

In [7]:
# 2) Funkcje pomocnicze
def sha256_file(p: Path, buf=1024*1024):
    h = hashlib.sha256()
    with p.open("rb") as f:
        while True:
            b = f.read(buf)
            if not b: break
            h.update(b)
    return h.hexdigest()

def img_info(p: Path):
    try:
        with Image.open(p) as im:
            return im.width, im.height
    except Exception:
        return None, None

def make_thumb(src: Path, dst_dir: Path, size=384):
    dst = dst_dir / src.name
    try:
        with Image.open(src) as im:
            im.thumbnail((size, size))
            im.convert("RGB").save(dst.with_suffix(".jpg"), quality=85)
    except Exception:
        pass

### 3. Czyszczenie staging przed nowym runem

In [8]:
### 3. Czyszczenie data/staging przed nowym runem

"""
    Czyści katalog data/staging przed nowym uruchomieniem ingestu.

    - Usuwa wszystkie podkatalogi RUN_ID oraz pliki w data/staging
    - Nie dotyka katalogu źródłowego z obrazami (dd_src.value)
    """

from pathlib import Path
import shutil

STAGING_ROOT = ROOT / "data" / "staging"

if STAGING_ROOT.exists():
    n_dirs = 0
    n_files = 0
    for p in STAGING_ROOT.iterdir():
        if p.is_dir():
            shutil.rmtree(p)
            n_dirs += 1
        elif p.is_file():
            p.unlink()
            n_files += 1
    print(f"Usunięto {n_dirs} katalogów i {n_files} plików z {STAGING_ROOT}")
else:
    print(f"Brak katalogu {STAGING_ROOT}, nic do sprzątnięcia.")

Usunięto 1 katalogów i 0 plików z /Users/olga/MetaLogic/data/staging


### 4. Główna procedura ingest

In [9]:
# 4) Główna procedura ingest (bez sieci)

from csv import DictWriter
from pathlib import Path
import shutil, time

assert Path(dd_src.value).exists(), f"Brak katalogu: {dd_src.value}"

records = []
exts = {".jpg", ".jpeg", ".png", ".tif", ".tiff", ".bmp", ".webp"}
src_dir = Path(dd_src.value)

# deterministycznie i z filtrem rozszerzeń
all_files = [p for p in src_dir.rglob("*") if p.is_file() and p.suffix]
files = [p for p in sorted(all_files) if p.suffix.lower() in exts][:it_max.value]
print(f"Znaleziono {len(files)} plików w {src_dir}")

n_small = 0
for p in files:
    if p.is_symlink():
        continue
    w, h = img_info(p)  # zakładam, że zwraca (w, h) lub (None, None) przy błędzie
    if not w or not h or w < MIN_W or h < MIN_H:
        n_small += 1
        continue

    rel = p.relative_to(src_dir)
    dst = DIR_STAGING / rel
    dst.parent.mkdir(parents=True, exist_ok=True)
    shutil.copy2(p, dst)

    rec = {
        "src_path": str(p),
        "rel_dst": str(rel),
        "size": p.stat().st_size,
        "sha256": sha256_file(p),
        "width": w,
        "height": h,
        "created_at": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime()),
    }
    records.append(rec)

    if cb_thumbs.value:
        make_thumb(dst, DIR_PREV)

print(f"Skopiowano {len(records)} plików do {DIR_STAGING}")
if n_small:
    print(f"Odrzucono przez wymiary: {n_small}")

Znaleziono 136 plików w /Users/olga/MetaLogic/inputs
Skopiowano 136 plików do /Users/olga/MetaLogic/data/staging/20260104_200942_A1


### 5. Sprawdzanie rozmiarów plików w katalogu staging

In [10]:
# 5) Sprawdzanie rozmiarów plików w katalogu staging

from pathlib import Path

sizes = [p.stat().st_size for p in (DIR_STAGING).rglob("*") if p.is_file()]
total_mb = sum(sizes) / (1024 * 1024)
avg_kb = (sum(sizes) / len(sizes)) / 1024 if sizes else 0

print(f"📦 Łączny rozmiar: {total_mb:.2f} MB")
print(f"📎 Średni rozmiar pliku: {avg_kb:.1f} KB")
print(f"📂 Liczba plików: {len(sizes)}")

📦 Łączny rozmiar: 125.57 MB
📎 Średni rozmiar pliku: 945.5 KB
📂 Liczba plików: 136


### 6. Zapis sources.tsv i sources.jsonl

In [11]:
# 6) Zapis sources.tsv i sources.jsonl

# --- Walidacja przed zapisem ---
if not records:
    print("⚠️ Brak rekordów do zapisania – folder inputs/ jest pusty.")
else:
    # zapis TSV
    tsv = DIR_STAGING / "sources.tsv"
    with tsv.open("w", encoding="utf-8", newline="") as f:
        writer = DictWriter(f, fieldnames=list(records[0].keys()), delimiter="\t")
        writer.writeheader()
        for r in records:
            writer.writerow(r)

    # zapis JSONL
    jsonl = DIR_STAGING / "sources.jsonl"
    with jsonl.open("w", encoding="utf-8") as f:
        for r in records:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

    print(f"✅ Zapisano {len(records)} rekordów do {DIR_STAGING}")

✅ Zapisano 136 rekordów do /Users/olga/MetaLogic/data/staging/20260104_200942_A1
